
<a href="https://colab.research.google.com/github/takzen/pytorch-black-belt/blob/main/28_Model_Surgery.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# 🥋 Lekcja 28: Model Surgery (Przeszczepianie Warstw)

Bierzemy gotowy model (`pretrained=True`), ale musimy go dostosować do naszych danych.

1.  **Head Replacement (Proste):** Podmieniamy ostatnią warstwę liniową (`fc` lub `classifier`), żeby zmienić liczbę klas.
2.  **Stem Replacement (Trudne):** Podmieniamy pierwszą warstwę konwolucyjną (`conv1`), żeby zmienić liczbę kanałów wejściowych.

**Black Belt Trick:**
Jeśli zmieniamy wejście z 3 kanałów (RGB) na 1 kanał (Grayscale), nie inicjalizujemy nowej warstwy losowo!
Bierzemy wagi z oryginalnej warstwy RGB, **uśredniamy je** i wkładamy do nowej warstwy.
Dzięki temu model od razu "umie" wykrywać krawędzie i kształty, zamiast uczyć się widzenia od nowa.

In [1]:
import torch
import torch.nn as nn
from torchvision import models

# 1. Pacjent: ResNet18 (Wytrenowany na ImageNet)
# W nowych wersjach torchvision używamy weights=...
model = models.resnet18(weights='IMAGENET1K_V1')

print("--- ORYGINAŁ ---")
print(f"Wejście (conv1): {model.conv1}")
print(f"Wyjście (fc):    {model.fc}")

--- ORYGINAŁ ---
Wejście (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
Wyjście (fc):    Linear(in_features=512, out_features=1000, bias=True)


## Operacja 1: Wymiana Głowy (Output Layer)

To standardowy Fine-Tuning.
ResNet18 ma na końcu warstwę `Linear(512, 1000)`.
My chcemy klasyfikować np. **2 klasy** (Kot vs Pies).

In [2]:
# Sprawdzamy ile cech wchodzi do ostatniej warstwy
in_features = model.fc.in_features

# Podmieniamy warstwę (Stara idzie do śmieci, nowa jest losowa)
model.fc = nn.Linear(in_features, 2)

print("--- PO WYMIANIE GŁOWY ---")
print(model.fc)
# Teraz model zwraca 2 logity zamiast 1000.

--- PO WYMIANIE GŁOWY ---
Linear(in_features=512, out_features=2, bias=True)


## Operacja 2: Wymiana Oczu (Input Layer) + Przeszczep Wag

To jest trudniejsze.
Mamy zdjęcia Rentgenowskie (1 kanał). ResNet chce 3 kanały.
Jeśli zrobimy `nn.Conv2d(1, 64, ...)`, nowa warstwa będzie losowa. Zniszczymy całą wiedzę o wykrywaniu krawędzi, którą ResNet zdobył na ImageNet.

**Trik:**
Wagi w `conv1` mają kształt `[64, 3, 7, 7]` (64 filtry, 3 kanały, kernel 7x7).
Możemy zsumować (lub uśrednić) te 3 kanały, żeby dostać `[64, 1, 7, 7]`.
To zadziała, bo krawędź na zdjęciu czarno-białym wygląda tak samo jak na kolorowym.

In [3]:
# 1. Zapisujemy starą warstwę
old_conv = model.conv1

# 2. Tworzymy nową warstwę (1 kanał wejściowy zamiast 3)
# Musimy zachować te same parametry (kernel, stride, padding, bias)
new_conv = nn.Conv2d(
    in_channels=1, 
    out_channels=old_conv.out_channels, 
    kernel_size=old_conv.kernel_size, 
    stride=old_conv.stride, 
    padding=old_conv.padding,
    bias=old_conv.bias is not None
)

print(f"Stare wagi: {old_conv.weight.shape}")
print(f"Nowe wagi (losowe): {new_conv.weight.shape}")

# 3. PRZESZCZEP WAG (Surgical Transplant)
with torch.no_grad():
    # Sumujemy wagi po wymiarze kanałów (dim=1) i dzielimy przez 3 (średnia)
    # [64, 3, 7, 7] -> [64, 1, 7, 7]
    weight_avg = old_conv.weight.mean(dim=1, keepdim=True)
    
    # Wstrzykujemy do nowej warstwy
    new_conv.weight.copy_(weight_avg)

# 4. Podmieniamy warstwę w modelu
model.conv1 = new_conv

print("\n✅ Przeszczep udany. Wagi z ImageNet zostały zachowane (jako Grayscale).")

Stare wagi: torch.Size([64, 3, 7, 7])
Nowe wagi (losowe): torch.Size([64, 1, 7, 7])

✅ Przeszczep udany. Wagi z ImageNet zostały zachowane (jako Grayscale).


In [4]:
# TEST ŻYWY
# Generujemy losowy obrazek w skali szarości (1 kanał)
dummy_xray = torch.randn(1, 1, 224, 224)

try:
    output = model(dummy_xray)
    print("\n--- TEST PRZEPŁYWU ---")
    print(f"Wejście: {dummy_xray.shape}")
    print(f"Wyjście: {output.shape} (Oczekiwane: [1, 2])")
    print("Pacjent przeżył operację.")
except Exception as e:
    print(f"💀 Błąd: {e}")


--- TEST PRZEPŁYWU ---
Wejście: torch.Size([1, 1, 224, 224])
Wyjście: torch.Size([1, 2]) (Oczekiwane: [1, 2])
Pacjent przeżył operację.


## 🥋 Black Belt Summary

1.  **Nie trenuj od zera**, jeśli nie musisz. Nawet jeśli masz inny rozmiar wejścia, możesz zaadaptować wagi.
2.  **Suma wag:** Jeśli zmieniasz wejście z 3 kanałów na 4 (np. RGB + Podczerwień), możesz wziąć wagi z RGB, a dla 4. kanału zainicjować zerami (lub średnią). Wtedy model na starcie działa jak zwykły ResNet, a z czasem uczy się używać podczerwieni.
3.  **Model Surgery** to codzienność w pracy z obrazami medycznymi i satelitarnymi.